<a href="https://colab.research.google.com/github/M0rph3e/ift6758-project-main/blob/fengineering2/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Motivation for the code and explanation [here](https://towardsdatascience.com/binary-classification-xgboost-hyperparameter-tuning-scenarios-by-non-exhaustive-grid-search-and-c261f4ce098d)

In [ ]:
#import comet_ml in the top of your file
from comet_ml import Experiment
import os
from copy import deepcopy
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import xgboost as xgb
random_state = 42

In [ ]:
COMET_API_KEY = 

#create an experiment with your api key
exp = Experiment(api_key=os.environ.get(COMET_API_KEY),
                 project_name='XGBoost',
                 auto_param_logging=False)

ValueError: ignored

In [ ]:
### Script to get the DataFrame and Split the data into train and test set for FE1

In [ ]:
# Apply XGB on distance and angle features from FE1
xgb1 = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='gpu_hist',
                          grow_policy='lossguide',
                          use_label_encoder=False,
                         random_state=random_state
                        )
xgb1.fit(X, y)

y_pred = xgb1.predict(X_test)
y_proba = xgb1.predict_proba(X_test)

In [ ]:
###Script to perfrom generation of Graphs

In [ ]:
### Script to get the DataFrame and Split the data into train and test set for FE2

In [ ]:
### Apply XGB on features from FE2
#dictionary for collecting results
results_dict = {}

#obtaining default parameters by calling .fit() to XGBoost model instance
xgb2 = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='gpu_hist',
                          grow_policy='lossguide',
                          use_label_encoder=False,
                         random_state=random_state
                        )
xgb2.fit(X, y)

#extracting default parameters from benchmark model
default_params = {}
gparams = xgb2.get_params()

#default parameters have to be wrapped in lists - even single values - so GridSearchCV can take them as inputs
for key in gparams.keys():
    gp = gparams[key]
    default_params[key] = [gp]

#benchmark model. Grid search is not performed, since only single values are provided as parameter grid.
#However, cross-validation is still executed
clf0 = GridSearchCV(estimator=xgb2, scoring='accuracy', param_grid=default_params, return_train_score=True, verbose=1, cv=3)
clf0.fit(X_train, y_train.values.ravel())

#results dataframe
df = pd.DataFrame(clf0.cv_results_)

#best parameters
bp = clf0.best_params_

#storing computed values in results dictionary
results_dict['xgb2'] = {'iterable_parameter': np.nan,
                         'classifier': deepcopy(clf0),
                         'cv_results': df.copy(),
                         'best_params': bp}



y_pred = clf0.predict(X_test)
y_proba = clf0.predict_proba(X_test)

In [ ]:
##Hyperparameter Tuning
#creating deepcopy of default parameters before manipulations
params = deepcopy(default_params)

#setting grid of selected parameters for iteration
param_grid = {'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200]}

#No. of jobs
bcvj = int(np.cumsum([len(x) for x in param_grid.values()])[-1])

#unwrapping list values of default parameters
default_params_xgb = {}

for key in default_params.keys():
    default_params_xgb[key] = default_params[key][0]

#providing default parameters to xgbc model, before randomized search cross-validation
xgb3 = xgb.XGBClassifier(**default_params_xgb)

clf = BayesSearchCV(estimator=xgb3, search_spaces=param_grid, n_iter=bcvj, scoring='accuracy', cv=3, return_train_score=True, verbose=3)
clf.fit(X_train, y_train.values.ravel())

#results dataframe
df = pd.DataFrame(clf.cv_results_)

#best parameters
bp = clf.best_params_

#storing computed values in results dictionary
results_dict['xgb3'] = {'classifier': deepcopy(clf),
                            'cv_results': df.copy(),
                            'best_params': bp}


y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [ ]:
###Script to perfrom generation of Graphs